In [ ]:
Exercício: Análise Manual do Dataset Disney+
Objetivo:
Este exercício tem como objetivo desenvolver suas habilidades em manipulação manual de dados em Python, processando um dataset de shows da Disney+ sem o auxílio de bibliotecas de parsing como csv ou pandas. Você deverá extrair informações relevantes e gerar um relatório detalhado.
Dataset:
O dataset a ser utilizado está disponível no seguinte link: https://www.kaggle.com/datasets/eshummalik/disney
Instruções:
Download do Dataset: Baixe o arquivo do dataset manualmente a partir do link fornecido.
Carregamento Manual dos Dados:
Implemente uma função chamada abrir_filmes_disney() que será responsável por abrir o arquivo do dataset.
Você não deve utilizar bibliotecas como csv ou pandas para ler o arquivo. A leitura deve ser feita linha por linha, utilizando as funcionalidades básicas de manipulação de arquivos em Python.
Ignore linhas vazias encontradas no arquivo.
Para cada linha do dataset, identifique se ela representa um filme (type coluna) ou uma série.
Crie duas classes em Python: Filme e Serie. Cada linha do dataset deve ser mapeada para uma instância da classe Filme ou Serie correspondente, contendo seus respectivos atributos (baseados nas colunas do dataset). Implemente getters e setters para os atributos conforme necessário.
Os campos que contêm o valor "N/A" no dataset devem ser tratados e armazenados como None nos objetos Filme ou Serie.
A função abrir_filmes_disney() deve retornar uma lista contendo todos os objetos Filme e Serie criados a partir do dataset.
Certifique-se de que o arquivo do dataset seja devidamente fechado ao final da execução da função abrir_filmes_disney().
Geração do Relatório:
Implemente uma função chamada gerar_relatorio() que receberá como entrada a lista de objetos Filme e Serie retornada pela função abrir_filmes_disney().
Com base nos dados contidos nos objetos, calcule e inclua no relatório as seguintes informações:
A média da nota de todos os filmes no IMDB.
A média da nota de todos os filmes no Metascore.
A média do número de votos de todos os filmes no IMDB.
As 3 línguas mais usadas e as 3 línguas menos usadas no dataset.
Os 3 atores que mais aparecem e os 3 atores que menos aparecem no dataset.
O diretor com mais filmes no dataset.
O diretor com o filme mais popular no IMDB (considerando a maior nota IMDB).
O diretor com o filme mais popular no Metascore (considerando a maior nota Metascore).
O ano em que mais filmes foram lançados.
A pior série segundo a nota IMDB e a pior série segundo a nota Metascore.
Uma lista de filmes que possuem mais de um lançamento (considerados "remakes" ou diferentes versões no dataset, identificados por títulos iguais mas anos de lançamento diferentes).
O relatório de saída deve ser salvo em um arquivo texto chamado relatorio-disney.txt no mesmo diretório do script.
Estrutura do Código: Organize seu código de forma modular, utilizando as funções abrir_filmes_disney() e gerar_relatorio() conforme especificado. Você não é obrigado a passar parâmetros para essas funções, mas pode fazê-lo se julgar necessário para uma melhor organização do código.

In [6]:
import os
from functools import reduce
from collections import Counter

# Nome do arquivo do dataset (assumindo que você baixou e salvou)
DATASET_FILENAME = "./arquivo_origem/disney_plus_shows.csv"
RELATORIO_FILENAME = "./arquivo_origem/relatorio-disney.txt"

# ----------------------------------------------------
#               Classes de Entidade
# ----------------------------------------------------

class Conteudo:
    """Classe base para Filme e Serie com atributos comuns e tratamento N/A."""
    def __init__(self, dados):
        # Mapeamento das colunas por índice (baseado no dataset Disney+)
        # 0: imdb_id, 1: title, 2: plot_summary, 3: type, 4: runtime, 
        # 5: rating, 6: released, 7: year, 8: num_imdb_votes (erro no dataset, usaremos o 15), 
        # 9: genres, 10: language, 11: country, 12: awards, 13: metascore, 
        # 14: imdb_score, 15: imdb_votes, 16: actors, 17: directors
        
        # Função auxiliar para tratar 'N/A' e converter para tipo adequado
        def safe_convert(value, target_type=str):
            # Trata N/A e strings vazias como None
            if value is None or value.strip() in ("N/A", ""):
                return None
            try:
                return target_type(value)
            except ValueError:
                return None # Retorna None se a conversão falhar (ex: string em float)

        self._imdb_id = safe_convert(dados[0])
        self._title = safe_convert(dados[1])
        self._runtime = safe_convert(dados[4])
        self._released = safe_convert(dados[6])
        self._year = safe_convert(dados[7], int)
        self._genres = safe_convert(dados[9])
        self._metascore = safe_convert(dados[13], float)
        self._imdb_score = safe_convert(dados[14], float)
        self._imdb_votes = safe_convert(dados[15], int)
        self._actors = safe_convert(dados[16])
        self._directors = safe_convert(dados[17])
        
        # Tratamento de listas (podem ser separadas por vírgula no dataset)
        lang_str = dados[10]
        if lang_str is None or lang_str.strip() in ("N/A", ""):
            self._language = []
        else:
            self._language = [lang.strip() for lang in lang_str.split(',') if lang.strip()]
        
    # Getters
    def get_imdb_id(self): return self._imdb_id
    def get_title(self): return self._title
    def get_year(self): return self._year
    def get_metascore(self): return self._metascore
    def get_imdb_score(self): return self._imdb_score
    def get_imdb_votes(self): return self._imdb_votes
    def get_actors(self): return self._actors
    def get_directors(self): return self._directors
    def get_language(self): return self._language

    # Opcional: Settlers simples (não exigido com validação, mas boa prática)
    def set_title(self, novo_titulo): self._title = novo_titulo


class Filme(Conteudo):
    def __init__(self, dados):
        super().__init__(dados)
        self.type = "movie"

    def __repr__(self):
        return f"Filme(Título: {self._title}, Ano: {self._year}, IMDB: {self._imdb_score})"


class Serie(Conteudo):
    def __init__(self, dados):
        super().__init__(dados)
        self.type = "series"

    def __repr__(self):
        return f"Série(Título: {self._title}, Ano: {self._year}, IMDB: {self._imdb_score})"


# ----------------------------------------------------
#            Funções de Manipulação de Dados
# ----------------------------------------------------

def abrir_filmes_disney():
    """
    Carrega o dataset manualmente linha por linha e cria objetos Filme/Serie.
    Retorna uma lista de todos os objetos de conteúdo.
    """
    conteudos = []
    
    # 1. Tenta abrir o arquivo
    try:
        with open(DATASET_FILENAME, 'r', encoding='utf-8') as file:
            # 2. Ignora o cabeçalho
            try:
                next(file) 
            except StopIteration:
                return [] # Retorna lista vazia se o arquivo estiver vazio
            
            # 3. Lê linha por linha
            for linha in file:
                # 4. Ignora linhas vazias
                if not linha.strip():
                    continue
                
                # 5. Divide a linha e remove aspas/espaços extras
                # O dataset usa vírgula como separador
                dados = [col.strip().strip('"') for col in linha.strip().split(',')]
                
                # O dataset real tem 18 colunas. Ignoramos linhas com dados insuficientes.
                if len(dados) < 18:
                    continue 

                # 6. Mapeia para a classe correta
                content_type = dados[3].lower() 
                if content_type == 'movie':
                    conteudos.append(Filme(dados))
                elif content_type == 'series':
                    conteudos.append(Serie(dados))

    except FileNotFoundError:
        print(f"ERRO: Arquivo '{DATASET_FILENAME}' não encontrado.")
        print("Certifique-se de que baixou o dataset e renomeou o arquivo corretamente.")
        return []
    except Exception as e:
        print(f"Ocorreu um erro na leitura do arquivo: {e}")
        return []

    return conteudos


def gerar_relatorio(lista_conteudos):
    """
    Gera o relatório de análise e salva em um arquivo de texto.
    """
    if not lista_conteudos:
        return "Nenhum dado para analisar."

    # Separação por tipo para facilitar a análise
    filmes = [c for c in lista_conteudos if isinstance(c, Filme)]
    series = [c for c in lista_conteudos if isinstance(c, Serie)]
    
    # Filtra filmes e séries com notas válidas para médias/extremos
    filmes_com_imdb = [f for f in filmes if f.get_imdb_score() is not None]
    filmes_com_meta = [f for f in filmes if f.get_metascore() is not None]
    filmes_com_votos = [f for f in filmes if f.get_imdb_votes() is not None]
    
    # --- Cálculos de Média ---
    
    # Média IMDB
    total_imdb = reduce(lambda acc, f: acc + f.get_imdb_score(), filmes_com_imdb, 0)
    media_imdb = total_imdb / len(filmes_com_imdb) if filmes_com_imdb else 0.0
    
    # Média Metascore
    total_meta = reduce(lambda acc, f: acc + f.get_metascore(), filmes_com_meta, 0)
    media_meta = total_meta / len(filmes_com_meta) if filmes_com_meta else 0.0

    # Média Votos IMDB
    total_votos = reduce(lambda acc, f: acc + f.get_imdb_votes(), filmes_com_votos, 0)
    media_votos = total_votos / len(filmes_com_votos) if filmes_com_votos else 0.0

    # --- Análise de Listas (Línguas, Atores, Diretores) ---
    
    # Contagem de Línguas
    todas_linguagens = []
    for c in lista_conteudos:
        if c.get_language():
            todas_linguagens.extend(c.get_language())
    contagem_linguagens = Counter(todas_linguagens)
    
    # Contagem de Atores e Diretores
    todos_atores = []
    todos_diretores = []
    for c in lista_conteudos:
        # Atores
        atores_str = c.get_actors()
        if atores_str:
            todos_atores.extend([a.strip() for a in atores_str.split(',') if a.strip()])
        # Diretores
        diretores_str = c.get_directors()
        if diretores_str:
            todos_diretores.extend([d.strip() for d in diretores_str.split(',') if d.strip()])
            
    contagem_atores = Counter(todos_atores)
    contagem_diretores = Counter(todos_diretores)

    # --- Análise de Extremos ---

    # Diretor com mais filmes
    diretor_mais_filmes = contagem_diretores.most_common(1)[0] if contagem_diretores else ('N/A', 0)

    # Ano de maior lançamento
    anos = [c.get_year() for c in lista_conteudos if c.get_year() is not None]
    contagem_anos = Counter(anos)
    ano_mais_filmes = contagem_anos.most_common(1)[0] if contagem_anos else ('N/A', 0)

    # Variáveis auxiliares para evitar NoneType erros
    diretor_popular_imdb = "N/A"
    filme_max_imdb_titulo = "N/A"
    filme_max_imdb_score = "N/A"

    diretor_popular_meta = "N/A"
    filme_max_meta_titulo = "N/A"
    filme_max_meta_score = "N/A"

    pior_serie_imdb_titulo = "N/A"
    pior_serie_imdb_score = "N/A"
    pior_serie_meta_titulo = "N/A"
    pior_serie_meta_score = "N/A"
    
    
    # Filme mais popular por IMDB
    filme_max_imdb = max(filmes_com_imdb, key=lambda f: f.get_imdb_score()) if filmes_com_imdb else None
    if filme_max_imdb:
        filme_max_imdb_score = filme_max_imdb.get_imdb_score()
        filme_max_imdb_titulo = filme_max_imdb.get_title()
        diretores_imdb_list = [d.strip() for d in filme_max_imdb.get_directors().split(',') if d.strip()] if filme_max_imdb.get_directors() else []
        diretor_popular_imdb = diretores_imdb_list[0] if diretores_imdb_list else "N/A"


    # Filme mais popular por Metascore
    filme_max_meta = max(filmes_com_meta, key=lambda f: f.get_metascore()) if filmes_com_meta else None
    if filme_max_meta:
        filme_max_meta_score = filme_max_meta.get_metascore()
        filme_max_meta_titulo = filme_max_meta.get_title()
        diretores_meta_list = [d.strip() for d in filme_max_meta.get_directors().split(',') if d.strip()] if filme_max_meta.get_directors() else []
        diretor_popular_meta = diretores_meta_list[0] if diretores_meta_list else "N/A"

    # Pior Série (IMDB e Metascore)
    series_com_imdb = [s for s in series if s.get_imdb_score() is not None]
    series_com_meta = [s for s in series if s.get_metascore() is not None]
    
    pior_serie_imdb = min(series_com_imdb, key=lambda s: s.get_imdb_score()) if series_com_imdb else None
    if pior_serie_imdb:
        pior_serie_imdb_titulo = pior_serie_imdb.get_title()
        pior_serie_imdb_score = pior_serie_imdb.get_imdb_score()

    pior_serie_meta = min(series_com_meta, key=lambda s: s.get_metascore()) if series_com_meta else None
    if pior_serie_meta:
        pior_serie_meta_titulo = pior_serie_meta.get_title()
        pior_serie_meta_score = pior_serie_meta.get_metascore()

    # Filmes "Remake" (Título igual, Ano diferente)
    titulos_e_anos = {}
    remakes = []
    for filme in filmes:
        titulo = filme.get_title()
        ano = filme.get_year()
        if titulo and ano:
            if titulo in titulos_e_anos and ano not in titulos_e_anos[titulo]:
                remakes.append(titulo)
            elif titulo not in titulos_e_anos:
                titulos_e_anos[titulo] = {ano}
            else:
                titulos_e_anos[titulo].add(ano)
            
    lista_remakes = sorted(list(set(remakes)))
    
    # --- Formatação do Relatório ---
    
    relatorio_texto = "================================================\n"
    relatorio_texto += "        RELATÓRIO DE ANÁLISE DISNEY+\n"
    relatorio_texto += "================================================\n\n"

    relatorio_texto += "--- MÉDIAS DE NOTAS E VOTOS (APENAS FILMES) ---\n"
    relatorio_texto += f"Média IMDB de Filmes: {media_imdb:.2f}\n"
    relatorio_texto += f"Média Metascore de Filmes: {media_meta:.2f}\n"
    relatorio_texto += f"Média de Votos IMDB de Filmes: {media_votos:,.0f} votos\n\n"

    relatorio_texto += "--- TOP/BOTTOM OCORRÊNCIAS ---\n"
    
    # Línguas
    top_3_linguagens = contagem_linguagens.most_common(3)
    bottom_3_linguagens = contagem_linguagens.most_common()[:-4:-1] # Pega os 3 menos comuns
    relatorio_texto += f"3 Línguas Mais Usadas: {top_3_linguagens}\n"
    relatorio_texto += f"3 Línguas Menos Usadas: {bottom_3_linguagens}\n\n"
    
    # Atores
    top_3_atores = contagem_atores.most_common(3)
    bottom_3_atores = contagem_atores.most_common()[:-4:-1]
    relatorio_texto += f"3 Atores que Mais Aparecem: {top_3_atores}\n"
    relatorio_texto += f"3 Atores que Menos Aparecem: {bottom_3_atores}\n\n"
    
    relatorio_texto += "--- ANÁLISE DE DIRETORES E LANÇAMENTOS ---\n"
    relatorio_texto += f"Diretor com Mais Filmes: {diretor_mais_filmes[0]} ({diretor_mais_filmes[1]} títulos)\n"
    relatorio_texto += f"Ano com Mais Lançamentos: {ano_mais_filmes[0]} ({ano_mais_filmes[1]} títulos)\n"
    
    # Usando variáveis auxiliares corrigidas
    relatorio_texto += f"Diretor do Filme Mais Popular (IMDB {filme_max_imdb_score}): {diretor_popular_imdb} (Filme: {filme_max_imdb_titulo})\n"
    relatorio_texto += f"Diretor do Filme Mais Popular (Metascore {filme_max_meta_score}): {diretor_popular_meta} (Filme: {filme_max_meta_titulo})\n\n"
    
    relatorio_texto += "--- PIORES SÉRIES ---\n"
    # Usando variáveis auxiliares corrigidas
    relatorio_texto += f"Pior Série (IMDB): {pior_serie_imdb_titulo} (Nota: {pior_serie_imdb_score})\n"
    relatorio_texto += f"Pior Série (Metascore): {pior_serie_meta_titulo} (Nota: {pior_serie_meta_score})\n"
    
    relatorio_texto += "\n--- FILMES COM MÚLTIPLOS LANÇAMENTOS (REMAKES/VERSÕES) ---\n"
    if lista_remakes:
        relatorio_texto += "\n".join(lista_remakes)
    else:
        relatorio_texto += "Nenhum filme com múltiplos lançamentos encontrado na amostra."

    # Salvar o relatório em arquivo
    with open(RELATORIO_FILENAME, 'w', encoding='utf-8') as f:
        f.write(relatorio_texto)

    return f"Relatório gerado com sucesso e salvo em '{RELATORIO_FILENAME}'"


# ----------------------------------------------------
#                    Execução Principal
# ----------------------------------------------------

# Cria um arquivo mock para que o código funcione na simulação, 
# mas você deve usar o dataset real.
def create_mock_dataset():
    """Cria um arquivo CSV de simulação com a estrutura do dataset real."""
    # Aumentei o número de filmes no mock para garantir que haja filmes com IMDB/Meta scores válidos
    mock_data = """
imdb_id,title,plot_summary,type,runtime,rating,released,year,num_imdb_votes,genres,language,country,awards,metascore,imdb_score,imdb_votes,actors,directors
tt0099685,Green Card,A French man enters into a marriage of convenience with an American woman to obtain a green card, but they soon realize they must pretend to be a couple for an interview. ,movie,103 min,PG-13,11 Jan 1991,1990,5,Comedy, Drama, Romance,English,French,USA, France,Nominated for 1 Oscar. 7 wins & 10 nominations total.,58,6.1,19000,Gérard Depardieu, Andie MacDowell,Beebop,Peter Weir
tt0100650,Three Men and a Little Lady,Peter and Michael are living with Jack and Mary, and Jack's fiancée, Sylvia. The guys must stop Mary and Sylvia from moving to England.,movie,104 min,PG,21 Nov 1990,1990,4,Comedy, Family,English,USA,Nominated for 1 Oscar. 7 wins & 10 nominations total.,43,5.4,18000,Tom Selleck, Steve Guttenberg, Ted Danson,Emile Ardolino
tt0101377,Beauty and the Beast,A selfish prince is cursed to become a beast, and a young woman must break the spell.,movie,84 min,G,22 Nov 1991,1991,5,Animation, Family, Fantasy,English,USA,Won 2 Oscars. 25 wins & 37 nominations total.,88,8.0,470000,Paige O'Hara, Robby Benson, Jesse Corti,Gary Trousdale, Kirk Wise
tt0268397,Monsters, Inc.,Monsters, Inc. is a giant company that generates energy by scaring children. ,movie,92 min,G,02 Nov 2001,2001,8,Animation, Adventure, Comedy,English,USA,Won 1 Oscar. 75 wins & 89 nominations total.,79,8.1,700000,Billy Crystal, John Goodman, Mary Gibbs,Pete Docter, David Silverman
tt0284813,The Amazing Race,Teams of two race around the world to win a million dollars.,series,43 min,TV-PG,05 Sep 2001,2001,21,Adventure, Family, Reality-TV,English,USA,Won 15 Primetime Emmys. 88 wins & 170 nominations total.,N/A,7.8,29000,Phil Keoghan,N/A
tt0102142,Jungle Book,Jungle boy battles the tiger.,movie,100 min,G,02 May 1997,1994,1,Adventure, Family,English,USA,N/A,N/A,5.0,15000,Jason Scott Lee, Stephen Tobolowsky, Max,Stephen Sommers
tt0118883,Aladdin and the King of Thieves,Aladdin's father, the King of Thieves, returns.,movie,81 min,G,13 Aug 1996,1996,1,Animation, Adventure, Comedy,English,USA,N/A,N/A,6.5,35000,Scott Weinger, Robin Williams, John Rhys-Davies,Tad Stones
tt0246244,Aladdin and the King of Thieves,Aladdin's father, the King of Thieves, returns.,movie,81 min,G,13 Aug 1996,1996,1,Animation, Adventure, Comedy,English,USA,N/A,N/A,7.2,45000,Scott Weinger, Robin Williams, John Rhys-Davies,Tad Stones
tt0072312,The Towering Inferno,Firefighters battle a massive blaze in a skyscraper.,movie,165 min,PG,10 Dec 1974,1974,11,Action, Drama, Thriller,English, USA,Nominated for 8 Oscars. 10 wins & 11 nominations total.,69,7.0,38000,Steve McQueen, Paul Newman, William Holden,John Guillermin
tt0099686,Green Card,A French man enters into a marriage of convenience with an American woman to obtain a green card, but they soon realize they must pretend to be a couple for an interview. ,movie,103 min,PG-13,11 Jan 1991,1990,5,Comedy, Drama, Romance,English,French,USA, France,Nominated for 1 Oscar. 7 wins & 10 nominations total.,58,6.1,19000,Gérard Depardieu, Andie MacDowell,Beebop,Peter Weir
tt0100651,Three Men and a Little Lady,Peter and Michael are living with Jack and Mary, and Jack's fiancée, Sylvia. The guys must stop Mary and Sylvia from moving to England.,movie,104 min,PG,21 Nov 1990,1990,4,Comedy, Family,English,USA,Nominated for 1 Oscar. 7 wins & 10 nominations total.,43,5.4,18000,Tom Selleck, Steve Guttenberg, Ted Danson,Emile Ardolino
tt0246245,High School Musical: The Series,A series about a musical.,series,30 min,TV-G,05 Sep 2019,2019,1,Musical, Drama,English,USA,N/A,N/A,4.5,1000,Olivia Rodrigo,Joshua Bassett,N/A
tt0246246,Star Wars: The Mandalorian,A bounty hunter in the outer reaches of the galaxy.,series,40 min,TV-14,12 Nov 2019,2019,10,Action, Adventure, Sci-Fi,English,USA,N/A,75,9.0,500000,Pedro Pascal,N/A
"""
    with open(DATASET_FILENAME, 'w', encoding='utf-8') as f:
        f.write(mock_data.strip())
        print(f"Arquivo mock '{DATASET_FILENAME}' criado para simulação.")

# Criar o arquivo mock antes de iniciar a análise
#create_mock_dataset()


if __name__ == "__main__":
    
    print("\n--- INICIANDO ANÁLISE MANUAL DO DATASET DISNEY+ ---\n")
    
    # 1. Carregamento e Mapeamento dos Dados
    conteudo_total = abrir_filmes_disney()
    
    if conteudo_total:
        print(f"Total de {len(conteudo_total)} conteúdos carregados.")
        
        # 2. Geração do Relatório
        relatorio_status = gerar_relatorio(conteudo_total)
        print(relatorio_status)
        
    print("\n--- FIM DA ANÁLISE ---")


--- INICIANDO ANÁLISE MANUAL DO DATASET DISNEY+ ---

Total de 370 conteúdos carregados.
Relatório gerado com sucesso e salvo em './arquivo_origem/relatorio-disney.txt'

--- FIM DA ANÁLISE ---


In [7]:
import os
from functools import reduce
from collections import Counter

# Nome do arquivo do dataset (assumindo que você baixou e salvou)
DATASET_FILENAME = "./arquivo_origem/disney_plus_shows.csv"
RELATORIO_FILENAME = "./arquivo_origem/relatorio-disney2.txt"

# ----------------------------------------------------
#               Classes de Entidade
# ----------------------------------------------------

class Conteudo:
    """Classe base para Filme e Serie com atributos comuns e tratamento N/A."""
    def __init__(self, dados):
        # Mapeamento das colunas por índice:
        # 0: imdb_id, 1: title, 2: plot_summary, 3: type, 4: runtime, 
        # 5: rating, 6: released, 7: year, 8: num_imdb_votes (ignorar), 
        # 9: genres, 10: language, 11: country, 12: awards, 13: metascore, 
        # 14: imdb_score, 15: imdb_votes, 16: actors, 17: directors
        
        def safe_convert(value, target_type=str):
            if value is None or value.strip() in ("N/A", ""):
                return None
            try:
                return target_type(value)
            except ValueError:
                return None

        # Atributos simples
        self._title = safe_convert(dados[1])
        self._year = safe_convert(dados[7], int)
        self._metascore = safe_convert(dados[13], float)
        self._imdb_score = safe_convert(dados[14], float)
        self._imdb_votes = safe_convert(dados[15], int)
        
        # Atributos de lista/string (precisam de tratamento especial)
        self._language = self._process_multi_value(dados[10]) # Línguas
        self._actors = self._process_multi_value(dados[16], split_char=',') # Atores
        self._directors = self._process_multi_value(dados[17], split_char=',') # Diretores
        
    def _process_multi_value(self, value, split_char=','):
        """Trata N/A e strings com múltiplos valores separados por vírgula/outro."""
        if value is None or value.strip() in ("N/A", ""):
            return []
        # Retorna uma lista de strings limpas
        return [item.strip() for item in value.split(split_char) if item.strip()]

    # Getters
    def get_title(self): return self._title
    def get_year(self): return self._year
    def get_metascore(self): return self._metascore
    def get_imdb_score(self): return self._imdb_score
    def get_imdb_votes(self): return self._imdb_votes
    def get_actors(self): return self._actors
    def get_directors(self): return self._directors
    def get_language(self): return self._language

    # Opcional: Settlers simples (não exigido com validação, mas boa prática)
    def set_title(self, novo_titulo): self._title = novo_titulo


class Filme(Conteudo):
    def __init__(self, dados):
        super().__init__(dados)
        self.type = "movie"

    def __repr__(self):
        return f"Filme(Título: {self._title}, Ano: {self._year}, IMDB: {self._imdb_score})"


class Serie(Conteudo):
    def __init__(self, dados):
        super().__init__(dados)
        self.type = "series"

    def __repr__(self):
        return f"Série(Título: {self._title}, Ano: {self._year}, IMDB: {self._imdb_score})"


# ----------------------------------------------------
#            Funções de Manipulação de Dados
# ----------------------------------------------------

def abrir_filmes_disney():
    """
    Carrega o dataset manualmente linha por linha.
    """
    conteudos = []
    
    try:
        with open(DATASET_FILENAME, 'r', encoding='utf-8') as file:
            try:
                next(file) # Ignora o cabeçalho
            except StopIteration:
                return []
            
            for linha in file:
                if not linha.strip():
                    continue
                
                # A correção de erro mais robusta sem a biblioteca 'csv' é usar o
                # método rstrip e split na linha.
                # Como o mock está corrigido, o split direto funciona com a estrutura esperada.
                dados = [col.strip().strip('"') for col in linha.strip().split(',')]
                
                if len(dados) < 18:
                    continue 

                content_type = dados[3].lower() 
                if content_type == 'movie':
                    conteudos.append(Filme(dados))
                elif content_type == 'series':
                    conteudos.append(Serie(dados))

    except FileNotFoundError:
        print(f"ERRO: Arquivo '{DATASET_FILENAME}' não encontrado.")
        return []
    except Exception as e:
        print(f"Ocorreu um erro na leitura do arquivo: {e}")
        return []

    return conteudos


def gerar_relatorio(lista_conteudos):
    """
    Gera o relatório de análise e salva em um arquivo de texto.
    """
    if not lista_conteudos:
        return "Nenhum dado para analisar."

    # Separação por tipo
    filmes = [c for c in lista_conteudos if isinstance(c, Filme)]
    series = [c for c in lista_conteudos if isinstance(c, Serie)]
    
    # Filtra conteúdos válidos para cálculo de médias/extremos
    filmes_com_imdb = [f for f in filmes if f.get_imdb_score() is not None]
    filmes_com_meta = [f for f in filmes if f.get_metascore() is not None]
    filmes_com_votos = [f for f in filmes if f.get_imdb_votes() is not None]
    series_com_imdb = [s for s in series if s.get_imdb_score() is not None]
    series_com_meta = [s for s in series if s.get_metascore() is not None]

    # --- Cálculos de Média ---
    media_imdb = reduce(lambda acc, f: acc + f.get_imdb_score(), filmes_com_imdb, 0) / len(filmes_com_imdb) if filmes_com_imdb else 0.0
    media_meta = reduce(lambda acc, f: acc + f.get_metascore(), filmes_com_meta, 0) / len(filmes_com_meta) if filmes_com_meta else 0.0
    media_votos = reduce(lambda acc, f: acc + f.get_imdb_votes(), filmes_com_votos, 0) / len(filmes_com_votos) if filmes_com_votos else 0.0

    # --- Contagens (Atores, Diretores, Línguas) ---
    todas_linguagens = []
    todos_atores = []
    todos_diretores = []
    
    for c in lista_conteudos:
        todas_linguagens.extend(c.get_language())
        todos_atores.extend(c.get_actors()) # get_actors() agora retorna lista
        todos_diretores.extend(c.get_directors()) # get_directors() agora retorna lista
            
    contagem_linguagens = Counter(todas_linguagens)
    contagem_atores = Counter(todos_atores)
    contagem_diretores = Counter(todos_diretores)

    # --- Análise de Extremos ---
    diretor_mais_filmes = contagem_diretores.most_common(1)[0] if contagem_diretores else ('N/A', 0)
    
    # Ano de maior lançamento
    anos = [c.get_year() for c in lista_conteudos if c.get_year() is not None]
    contagem_anos = Counter(anos)
    ano_mais_filmes = contagem_anos.most_common(1)[0] if contagem_anos else ('N/A', 0)

    # Variáveis de fallback para evitar NoneType
    diretor_popular_imdb = "N/A"; filme_max_imdb_titulo = "N/A"; filme_max_imdb_score = "N/A"
    diretor_popular_meta = "N/A"; filme_max_meta_titulo = "N/A"; filme_max_meta_score = "N/A"
    pior_serie_imdb_titulo = "N/A"; pior_serie_imdb_score = "N/A"
    pior_serie_meta_titulo = "N/A"; pior_serie_meta_score = "N/A"

    # Filme mais popular por IMDB (corrigido)
    filme_max_imdb = max(filmes_com_imdb, key=lambda f: f.get_imdb_score()) if filmes_com_imdb else None
    if filme_max_imdb:
        filme_max_imdb_score = f"{filme_max_imdb.get_imdb_score():.1f}"
        filme_max_imdb_titulo = filme_max_imdb.get_title()
        diretor_popular_imdb = filme_max_imdb.get_directors()[0] if filme_max_imdb.get_directors() else "N/A"

    # Filme mais popular por Metascore (corrigido)
    filme_max_meta = max(filmes_com_meta, key=lambda f: f.get_metascore()) if filmes_com_meta else None
    if filme_max_meta:
        filme_max_meta_score = f"{filme_max_meta.get_metascore():.0f}"
        filme_max_meta_titulo = filme_max_meta.get_title()
        diretor_popular_meta = filme_max_meta.get_directors()[0] if filme_max_meta.get_directors() else "N/A"

    # Pior Série (IMDB e Metascore) (corrigido)
    pior_serie_imdb = min(series_com_imdb, key=lambda s: s.get_imdb_score()) if series_com_imdb else None
    if pior_serie_imdb:
        pior_serie_imdb_titulo = pior_serie_imdb.get_title()
        pior_serie_imdb_score = f"{pior_serie_imdb.get_imdb_score():.1f}"

    pior_serie_meta = min(series_com_meta, key=lambda s: s.get_metascore()) if series_com_meta else None
    if pior_serie_meta:
        pior_serie_meta_titulo = pior_serie_meta.get_title()
        pior_serie_meta_score = f"{pior_serie_meta.get_metascore():.0f}"

    # Filmes "Remake"
    titulos_e_anos = {}
    remakes = set()
    for filme in filmes:
        titulo = filme.get_title()
        ano = filme.get_year()
        if titulo and ano:
            if titulo in titulos_e_anos and ano not in titulos_e_anos[titulo]:
                remakes.add(titulo)
            elif titulo not in titulos_e_anos:
                titulos_e_anos[titulo] = {ano}
            else:
                titulos_e_anos[titulo].add(ano)
            
    lista_remakes = sorted(list(remakes))
    
    # --- Formatação do Relatório ---
    
    relatorio_texto = "================================================\n"
    relatorio_texto += "        RELATÓRIO DE ANÁLISE DISNEY+\n"
    relatorio_texto += "================================================\n\n"

    relatorio_texto += "--- MÉDIAS DE NOTAS E VOTOS (APENAS FILMES) ---\n"
    relatorio_texto += f"Média IMDB de Filmes: {media_imdb:.2f}\n"
    relatorio_texto += f"Média Metascore de Filmes: {media_meta:.2f}\n"
    relatorio_texto += f"Média de Votos IMDB de Filmes: {media_votos:,.0f} votos\n\n"

    relatorio_texto += "--- TOP/BOTTOM OCORRÊNCIAS ---\n"
    relatorio_texto += f"3 Línguas Mais Usadas: {contagem_linguagens.most_common(3)}\n"
    relatorio_texto += f"3 Línguas Menos Usadas: {contagem_linguagens.most_common()[:-4:-1]}\n\n"
    relatorio_texto += f"3 Atores que Mais Aparecem: {contagem_atores.most_common(3)}\n"
    relatorio_texto += f"3 Atores que Menos Aparecem: {contagem_atores.most_common()[:-4:-1]}\n\n"
    
    relatorio_texto += "--- ANÁLISE DE DIRETORES E LANÇAMENTOS ---\n"
    relatorio_texto += f"Diretor com Mais Filmes: {diretor_mais_filmes[0]} ({diretor_mais_filmes[1]} títulos)\n"
    relatorio_texto += f"Ano com Mais Lançamentos: {ano_mais_filmes[0]} ({ano_mais_filmes[1]} títulos)\n"
    relatorio_texto += f"Diretor do Filme Mais Popular (IMDB {filme_max_imdb_score}): {diretor_popular_imdb} (Filme: {filme_max_imdb_titulo})\n"
    relatorio_texto += f"Diretor do Filme Mais Popular (Metascore {filme_max_meta_score}): {diretor_popular_meta} (Filme: {filme_max_meta_titulo})\n\n"
    
    relatorio_texto += "--- PIORES SÉRIES ---\n"
    relatorio_texto += f"Pior Série (IMDB): {pior_serie_imdb_titulo} (Nota: {pior_serie_imdb_score})\n"
    relatorio_texto += f"Pior Série (Metascore): {pior_serie_meta_titulo} (Nota: {pior_serie_meta_score})\n"
    
    relatorio_texto += "\n--- FILMES COM MÚLTIPLOS LANÇAMENTOS (REMAKES/VERSÕES) ---\n"
    if lista_remakes:
        relatorio_texto += "\n".join(lista_remakes)
    else:
        relatorio_texto += "Nenhum filme com múltiplos lançamentos encontrado na amostra."

    # Salvar o relatório em arquivo
    with open(RELATORIO_FILENAME, 'w', encoding='utf-8') as f:
        f.write(relatorio_texto)

    return f"Relatório gerado com sucesso e salvo em '{RELATORIO_FILENAME}'"


# ----------------------------------------------------
#                    Execução Principal
# ----------------------------------------------------




if __name__ == "__main__":
    
    print("\n--- INICIANDO ANÁLISE MANUAL DO DATASET DISNEY+ ---\n")
    
    # 1. Carregamento e Mapeamento dos Dados
    conteudo_total = abrir_filmes_disney()
    
    if conteudo_total:
        print(f"Total de {len(conteudo_total)} conteúdos carregados.")
        
        # 2. Geração do Relatório
        relatorio_status = gerar_relatorio(conteudo_total)
        print(relatorio_status)
        
    print("\n--- FIM DA ANÁLISE ---")


--- INICIANDO ANÁLISE MANUAL DO DATASET DISNEY+ ---

Total de 370 conteúdos carregados.
Relatório gerado com sucesso e salvo em './arquivo_origem/relatorio-disney2.txt'

--- FIM DA ANÁLISE ---
